In [54]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import nltk
from sklearn.utils import shuffle
from nlpaug.augmenter.word import SynonymAug
import nlpaug.augmenter.word as naw
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, KeyedVectors
from gensim.utils import simple_preprocess
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [56]:
dataset = pd.read_csv("en_dataset_with_stop_words.csv")

In [57]:
dataset = dataset.drop(columns = ["directness","HITId","annotator_sentiment","target","group"])

In [58]:
def label(x):
    if "hateful" in x:
        return 1
    elif "normal" in x:
        return 0
    else:
        return -2

In [59]:
dataset["sentiment"] = dataset["sentiment"].apply(lambda x : label(x))

In [60]:
dataset = dataset.rename(columns = {"sentiment" : "Hate_Speech"})

In [61]:
dataset = dataset.drop(dataset[dataset["Hate_Speech"] == -2].index)

In [ ]:
dataset["Hate_Speech"].value_counts()

In [64]:
import re
import string
punch = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'

def data_preprocessing(tweet):
    tweet = re.sub(r'&gt', '', tweet)
    tweet = re.sub(r'\b\\x\S\S', '',tweet)
    tweet = re.sub(r'\\u\S*', '', tweet)
    tweet = re.sub(r'\B\\x\S+', '',tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'ð', '', tweet)
    tweet ="".join([i for i in tweet if i not in punch])
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub('\n','',tweet)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','', tweet)
    tweet = re.sub('  +', '', tweet)
    tweet = re.sub(r'pic.twitter.com.[\w]+', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'\burl', '', tweet)
    tweet = re.sub(r'@', '', tweet)
    tweet = re.sub(r'\A', '', tweet)
    tweet = re.sub('user','', tweet)
    # tweet = re.sub(r'\s', '', tweet)
    if tweet.startswith("rt user"): return False
    return tweet

from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)


    if None in corrected_text:
        return text      
    
    return " ".join(corrected_text)  

<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\User\AppData\Local\Temp\ipykernel_9824\69068997.py:15: SyntaxWarning: invalid escape sequence '\.'
  tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','', tweet)


In [65]:
dataset['tweet'] = dataset['tweet'].apply(data_preprocessing)
dataset['tweet'] = dataset['tweet'].apply(correct_spellings)

In [267]:
# testing.to_csv('ready_to_translate.csv')

In [66]:
# Importing dataset that is ready to translate
import pandas as pd
englishDS = pd.read_csv("ready_to_translate.csv",index_col= 0)
englishDS = englishDS.dropna(axis=0)

In [67]:
englishDS['Hate_Speech'].value_counts()

Hate_Speech
1    1278
0     974
Name: count, dtype: int64

In [71]:
# NLP AUG to balance the dataset using wordnet

def augment_text(df,samples=304,pr=0.2):
    aug = naw.SynonymAug(aug_src='wordnet')

    new_text=[]

    df_n=df[df["Hate_Speech"]==0].reset_index(drop=True)

    for i in np.random.randint(0,len(df_n),samples):

            text = df_n.iloc[i]['tweet']
            augmented_text = aug.augment(text)
            new_text.append(augmented_text)

    ## dataframe
    new=pd.DataFrame({'tweet':new_text,'Hate_Speech':0})


    df = pd.concat([df, new],axis=0).reset_index(drop=True)
    df = shuffle(df)
    return df

train = augment_text(englishDS)

In [85]:
train['Hate_Speech'].value_counts()

Hate_Speech
0    1278
1    1278
Name: count, dtype: int64

In [4]:
# Reading the dataset that has been translated

IDT = pd.read_csv("IndonesianDatasetWOPP.csv")
IDT = IDT.drop(columns="Unnamed: 0")
IDT = IDT.dropna()

In [6]:
# Removing Stopwords

stop_words = pd.read_json('stopwords-id.json')
stop_words.columns = ['Words']
stop_words['Words'].head()
stopwords = stop_words['Words'].tolist()
def remove_stopwords(tweet):
    tweet_tokens = word_tokenize(tweet)
    filtered_tweets = [w for w in tweet_tokens if not w in stopwords]
    return " ".join(filtered_tweets)

IDT.translated_tweet = IDT['translated_tweet'].apply(remove_stopwords)


In [7]:
# Tokenizing the tweets

tokenized_data = IDT
tokenized_data['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in tokenized_data['translated_tweet']] 

0    [dia, lucu, lapisan, gulanya, standar, suka, m...
1    [benar, benar, benar, cauline, hansom, apa, pu...
2                  [anda, menghormati, negro, bendera]
3                     [sh, wanita, benar, terbelakang]
4       [pertanyaanmu, faggot, homgote, sialan, fagot]
5                            [malam, buruk, mongoloid]
6    [saya, berharap, salah, memerintah, dunia, trump]
7                               [susan, anda, berasal]
8                                [kami, benar, vagina]
9                              [beri, memanggil, spic]
Name: tokenized_text, dtype: object


In [ ]:
# Training the word2vec model

size = 1000
window = 3
min_count = 1
workers = 3
sg = 1

word2vec_model_file = 'C:\\Users\\User\\Documents\\Pythobn\\RM\\' + 'word2vec_' + str(size) + '.model'
start_time = time.time()
tokenized_tweets = pd.Series(tokenized_data['tokenized_text']).values
w2v_model = Word2Vec(tokenized_tweets, min_count=min_count, vector_size=size, workers=workers, window=window, sg=sg)
print(f"Time taken to train word2Vec model: {time.time() - start_time}")
w2v_model.save(word2vec_model_file)

In [ ]:
# Loading the word2vec model

size = 1000
window = 3
min_count = 1
workers = 3
sg = 1
word2vec_model_file = 'C:\\Users\\User\\Documents\\Pythobn\\RM\\' + 'word2vec_' + str(size) + '.model'
sg_w2v_model = Word2Vec.load(word2vec_model_file)
vocab = sg_w2v_model.wv.key_to_index
print(sg_w2v_model.wv.get_vector(vocab['indonesia']))

In [10]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [sg_w2v_model.wv[word] for word in words if word in sg_w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(1000)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

In [11]:
# Reading the dataset that has been translated and preprocessed

df = pd.read_csv('translated_dataset.csv', index_col=0)

In [12]:
df['Hate_Speech'].value_counts()

Hate_Speech
1    1276
0    1273
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['translated_tweet'], df['Hate_Speech'], test_size=0.2, random_state=42)

In [14]:
import numpy as np

X_train = np.array([vectorize(str(sentence)) for sentence in X_train])
X_test = np.array([vectorize(str(sentence)) for sentence in X_test])

In [16]:
clf_lr = LogisticRegression(random_state=42)
clf_lr.fit(X_train, y_train)
y_pred = clf_lr.predict(X_test)
print(classification_report(y_test, y_pred))
print(classification_report(y_train, clf_lr.predict(X_train)))

              precision    recall  f1-score   support

           0       0.55      0.62      0.59       256
           1       0.57      0.50      0.53       254

    accuracy                           0.56       510
   macro avg       0.56      0.56      0.56       510
weighted avg       0.56      0.56      0.56       510

              precision    recall  f1-score   support

           0       0.55      0.61      0.58      1017
           1       0.56      0.50      0.53      1022

    accuracy                           0.55      2039
   macro avg       0.55      0.55      0.55      2039
weighted avg       0.55      0.55      0.55      2039



In [17]:
clf_dt = DecisionTreeClassifier(random_state=42)
clf_dt.fit(X_train, y_train)
y_pred = clf_dt.predict(X_test)
print(classification_report(y_test, y_pred))
print(classification_report(y_train, clf_dt.predict(X_train)))

              precision    recall  f1-score   support

           0       0.53      0.58      0.56       256
           1       0.53      0.48      0.51       254

    accuracy                           0.53       510
   macro avg       0.53      0.53      0.53       510
weighted avg       0.53      0.53      0.53       510

              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1017
           1       0.97      0.93      0.95      1022

    accuracy                           0.95      2039
   macro avg       0.95      0.95      0.95      2039
weighted avg       0.95      0.95      0.95      2039



In [18]:
clf_svm = svm.SVC(kernel='linear',random_state=42)
clf_svm.fit(X_train, y_train)
y_pred = clf_svm.predict(X_test)
print(classification_report(y_test, y_pred))
print(classification_report(y_train, clf_svm.predict(X_train)))

              precision    recall  f1-score   support

           0       0.51      0.87      0.64       256
           1       0.55      0.16      0.25       254

    accuracy                           0.52       510
   macro avg       0.53      0.51      0.45       510
weighted avg       0.53      0.52      0.45       510

              precision    recall  f1-score   support

           0       0.52      0.85      0.65      1017
           1       0.59      0.22      0.32      1022

    accuracy                           0.53      2039
   macro avg       0.56      0.53      0.48      2039
weighted avg       0.56      0.53      0.48      2039



In [19]:
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(X_train, y_train)
y_pred = clf_knn.predict(X_test)
print(classification_report(y_test, y_pred))
print(classification_report(y_train, clf_knn.predict(X_train)))

              precision    recall  f1-score   support

           0       0.59      0.50      0.54       256
           1       0.56      0.65      0.60       254

    accuracy                           0.57       510
   macro avg       0.58      0.57      0.57       510
weighted avg       0.58      0.57      0.57       510

              precision    recall  f1-score   support

           0       0.78      0.69      0.73      1017
           1       0.72      0.80      0.76      1022

    accuracy                           0.75      2039
   macro avg       0.75      0.75      0.75      2039
weighted avg       0.75      0.75      0.75      2039



In [20]:
clf_rf = RandomForestClassifier(random_state=42, n_jobs=200)
clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(classification_report(y_train, clf_rf.predict(X_train)))

              precision    recall  f1-score   support

           0       0.55      0.64      0.59       256
           1       0.56      0.46      0.51       254

    accuracy                           0.55       510
   macro avg       0.56      0.55      0.55       510
weighted avg       0.56      0.55      0.55       510

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      1017
           1       0.97      0.93      0.95      1022

    accuracy                           0.95      2039
   macro avg       0.95      0.95      0.95      2039
weighted avg       0.95      0.95      0.95      2039



In [21]:
# Cross validation with the other model that has been trained with the Originally Indonesian dataset.

IndoDT = pd.read_csv('cleaned_tweet.csv')

In [22]:
neg = IndoDT[IndoDT['Hate_Speech'] == 0].sample(500,random_state=42)
pos = IndoDT[IndoDT['Hate_Speech'] == 1].sample(500,random_state=42)

In [32]:
test_df = pd.concat([neg, pos])
test_df = shuffle(test_df)
test_df = test_df.dropna()

In [33]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 9106 to 7779
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Tweet           998 non-null    object
 1   Hate_Speech     998 non-null    int64 
 2   tokenized_text  998 non-null    object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


In [35]:
X_test = test_df['Tweet']
X_test = np.array([vectorize(str(sentence)) for sentence in X_test])
y_test = test_df['Hate_Speech']

In [36]:
y_pred = clf_lr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.93      0.65       498
           1       0.56      0.09      0.15       500

    accuracy                           0.51       998
   macro avg       0.53      0.51      0.40       998
weighted avg       0.53      0.51      0.40       998



In [37]:
y_pred = clf_knn.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.56      0.55       498
           1       0.55      0.53      0.54       500

    accuracy                           0.55       998
   macro avg       0.55      0.55      0.55       998
weighted avg       0.55      0.55      0.55       998



In [38]:
y_pred = clf_svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.98      0.66       498
           1       0.58      0.03      0.05       500

    accuracy                           0.50       998
   macro avg       0.54      0.50      0.36       998
weighted avg       0.54      0.50      0.36       998



In [39]:
y_pred = clf_dt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.57      0.52       498
           1       0.48      0.40      0.44       500

    accuracy                           0.48       998
   macro avg       0.48      0.48      0.48       998
weighted avg       0.48      0.48      0.48       998



In [40]:
y_pred = clf_rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.51      0.82      0.63       498
           1       0.55      0.21      0.31       500

    accuracy                           0.52       998
   macro avg       0.53      0.52      0.47       998
weighted avg       0.53      0.52      0.47       998

